In [1]:
from keras.applications import vgg16
import numpy as np 
import matplotlib.pyplot as plt
import cv2 as cv 
import os 
from sklearn import preprocessing
from random import shuffle
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Activation, Dropout, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from random import randint
import h5py
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.callbacks import TensorBoard
from keras.models import Model
import time
import pickle

In [2]:
def resize_image(img_array, height, width):
    return cv.resize(img_array, (height, width))

def plot_image(img_array):
    plt.imshow(img_array, cmap='gray')
    plt.show()

# Seperating features from the labels
def sep_features_labels(data):
    X = []
    y = []
    for features, label in data:
        X.append(features)
        y.append(label)
    return X, y

#Reshaping the features 
def reshape(X, y, img_width, img_height):
    X = np.array(X).reshape(-1, img_height, img_width, 1)
    X = np.array(X).astype('float')/255.0
    # y_train = preprocessing.LabelEncoder().fit_transform(y_train)
    y = np.asarray(y).astype('float32').reshape((-1,1))
    
    return X, y

def pickle_save(x_name, y_name, X, y):
    pickle_out = open(f'{x_name}.pickle', "wb")
    pickle.dump(X, pickle_out)
    pickle_out.close()

    pickle_out = open(f'{y_name}.pickle', 'wb')
    pickle.dump(y, pickle_out)
    pickle_out.close()

def pickle_load(x_name, y_name):    
    pickle_in = open(x_name, 'rb')
    X = pickle.load(pickle_in)
    pickle_in.close()

    pickle_in = open(y_name, 'rb')
    y = pickle.load(pickle_in)
    pickle_in.close()
    return X, y

In [4]:
img_height = 120
img_width = 50
classes = ['0', '1']

In [3]:
X_train, y_train = pickle_load('/content/drive/MyDrive/Shared Datasets/Loaded data (binary aug)/X_train.pickle', '/content/drive/MyDrive/Shared Datasets/Loaded data (binary aug)/y_train.pickle')
X_val, y_val = pickle_load('//content/drive/MyDrive/Shared Datasets/Loaded data (binary aug)/X_val.pickle', '/content/drive/MyDrive/Shared Datasets/Loaded data (binary aug)/y_val.pickle')
X_test, y_test = pickle_load('/content/drive/MyDrive/Shared Datasets/Loaded data (binary aug)/X_test.pickle', '/content/drive/MyDrive/Shared Datasets/Loaded data (binary aug)/y_test.pickle')

In [5]:
X_train=np.dstack([X_train] * 3)
X_val =np.dstack([X_val ]*3)
X_test =np.dstack([X_test]*3)

X_train = X_train.reshape(-1, img_height,img_width,3)
X_val= X_val.reshape (-1,img_height,img_width,3)
X_test= X_test.reshape (-1,img_height,img_width,3)


X_train.shape, X_val.shape, X_test.shape

((20992, 120, 50, 3), (5998, 120, 50, 3), (3000, 120, 50, 3))

In [13]:
y_train.shape, y_val.shape, y_test.shape

((20992, 1), (5998, 1), (3000, 1))

In [15]:
vgg = vgg16.VGG16(weights='imagenet', 
                  include_top=False, 
                  input_shape=(img_height, img_width, 3))

for layer in vgg.layers:
  layer.trainable = False

feature_ex = vgg.predict(X_train)
features = feature_ex.reshape(feature_ex.shape[0], -1)

for i, layer in enumerate(vgg.layers):
  print(f'{i} {layer.__class__.__name__}, {layer.trainable}')

0 InputLayer, False
1 Conv2D, False
2 Conv2D, False
3 MaxPooling2D, False
4 Conv2D, False
5 Conv2D, False
6 MaxPooling2D, False
7 Conv2D, False
8 Conv2D, False
9 Conv2D, False
10 MaxPooling2D, False
11 Conv2D, False
12 Conv2D, False
13 Conv2D, False
14 MaxPooling2D, False
15 Conv2D, False
16 Conv2D, False
17 Conv2D, False
18 MaxPooling2D, False


In [14]:
feature_ex = vgg.predict(X_val)
val_features = feature_ex.reshape(feature_ex.shape[0], -1)

ValueError: ignored

In [11]:
model = Sequential()
model.add(Dense(256, activation='relu', input_dim=4 * 4 * 512))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=["accuracy"])

history = model.fit(features, y_train,
epochs=30,
batch_size=20,
validation_data=(val_features, y_val))

Epoch 1/30


ValueError: ignored

In [20]:
def lw(bottom_model, num_classes):
    """creates the top or head of the model that will be 
    placed ontop of the bottom layers"""

    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(512, activation='relu')(top_model)
    # top_model = Dropout(0.5)(top_model)
    top_model = Dense(num_classes, activation='sigmoid')(top_model)
    return top_model 

In [21]:
num_classes = 1

FC_Head = lw(vgg, num_classes)

model = Model(inputs = vgg.input, outputs = FC_Head)

In [18]:
print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 120, 50, 3)]      0         
                                                                 
 block1_conv1 (Conv2D)       (None, 120, 50, 64)       1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 120, 50, 64)       36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 60, 25, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 60, 25, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 60, 25, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 30, 12, 128)       0     

In [22]:
model.compile(optimizer='rmsprop', loss = 'binary_crossentropy',metrics = ['accuracy'])
history = model.fit(X_train,y_train, 
                    epochs=30, batch_size=132,
                    validation_data=(X_val, y_val),
                    verbose = 1)

Epoch 1/30
160/160 [==============================] - 11s 65ms/step - loss: 0.5598 - accuracy: 0.7212 - val_loss: 0.5643 - val_accuracy: 0.6826
Epoch 2/30
160/160 [==============================] - 10s 61ms/step - loss: 0.4692 - accuracy: 0.7609 - val_loss: 0.4286 - val_accuracy: 0.7786
Epoch 3/30
160/160 [==============================] - 10s 61ms/step - loss: 0.4409 - accuracy: 0.7722 - val_loss: 0.4371 - val_accuracy: 0.7793
Epoch 4/30
160/160 [==============================] - 10s 61ms/step - loss: 0.4267 - accuracy: 0.7843 - val_loss: 0.4179 - val_accuracy: 0.7928
Epoch 5/30
160/160 [==============================] - ETA: 0s - loss: 0.4141 - accuracy: 0.7908

KeyboardInterrupt: ignored